# Two Sigma: Using News to Predict Stock Movements

# 前書き

ニュース分析の内容を使って株価のパフォーマンスを予測できますか？  
今日のデータの偏在により、あらゆる規模の投資家がより良い投資判断を下すことができます。  
課題は、どのデータが有用であるかを決定するためにデータを取り込み、解釈し、この情報の海で信号を見つけることです。  
2つのシグマはこの挑戦に情熱を燃やしていて、カグルコミュニティと共有することに興奮しています。  

科学的に牽引された投資マネージャーとして、Two Sigmaは17年以上にわたりテクノロジーとデータ科学を財務予測に適用してきました。  
ビッグデータ、AI、機械学習の先駆的な進歩により、投資業界は前進しました。  
今、革新の継続的な追求にKagglersと熱心に取り組んでいます。  

株価を予測するためにニュースデータを分析することによって、カグラーズはニュースの予測力を理解する研究の状態を前進させるユニークな機会を得ます。  
この力を活用すれば、財政的な成果を予測し、世界中に大きな経済的影響をもたらすのに役立ちます。  

このコンテストのデータは、次のソースから得られます。  

Intrinioが提供する市場データ。  
Thomson Reutersが提供するニュースデータ。著作権©、トムソン・ロイター、2017年。All Rights Reserved。このサービスの使用、複製、または販売、またはここに含まれるデータは、競技規則に記載されている場合を除き、厳重に禁止されています。  
  
トムソン・ロイター・キネシス・ロゴおよびトムソン・ロイターは、トムソン・ロイターおよびその関連会社の米国およびその他の国における商標または登録商標です。

## 問題定義
今回はニュース記事から株価がどのように推移するかを予測する問題です。

## カーネルの進め方
1.データのインポート  
2.データ探索  
3.データの整頓  
4.使用モデルの検証  
5.学習データの作成  
6.学習  
7.予測  

# ライブラリのインポート

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# データのインポート

この大会では、2つのデータソースに基づいて将来の株価収益率を予測します。

イントリニオが提供する市場データ（2007年〜現在） - 開始価格、終値、取引量、計算されたリターンなどの金融市場情報を含んでいます。  
ニュースデータ（2007年〜現在）出典：Thomson Reuters  - 記事の詳細、感情、およびその他の解説など、資産に関するニュース記事/アラートに関する情報が含まれています。  
各アセットはassetCodeによって識別されます（1つの企業に複数のアセットコードがあることに注意してください）。あなたがやりたいことに応じて、市場データをニュースデータに結合する方法として、assetCode、assetName、またはtimeを使用することができます。  

これはカーネルのみの時間ベースの競争であるため、標準的なKaggleの競技会と同じように、データファイルと直接対話することはありません。データをフェッチして予測を行う方法の詳細については、提出の手順を参照してください。指示に記載されているように、競合データ内の将来の合成データに遭遇します。これは、実際の将来のデータが導入する量、タイムライン、および計算上の負担をシミュレートするために含まれています。  

また、カスタムPythonモジュールを使用すると、参加する必要のあるステップを簡単に把握でき、いつどのアセットコードを予測するのか、エクステンションごとに市場取引日を示すことができます。ステージ1では、リーダーボードは2017-01-01から2018-07-31までの過去の期間のパフォーマンスを表示します。ステージ2では、Kaggleは約6ヶ月の将来のデータで参加者の選択したカーネルを再実行します。  

データは、Kernels環境でPandasデータフレームとして格納され、取り出されます。列型は、メモリ内のスペースを最小限に抑えるように最適化されています。

市場データ  
データには、米国に上場されている計測器のサブセットが含まれています。含まれる商品のセットは毎日変更され、取引される金額と情報の可用性に基づいて決定されます。これは、このデータのサブセットに出入りする機器が存在する可能性があることを意味します。したがって、提供されるデータにはギャップが存在する可能性があり、必ずしもそのデータが存在しないことを意味するものではありません（選択基準のために行が含まれていない可能性が高い）。  

市場データには、異なるタイムスパンで計算されたさまざまなリターンが含まれています。この市場データセットのすべてのリターンには、次のプロパティがあります。  

リターンは常にオープン・ツー・オープン（1取引日のオープン時から別のオープン時）またはクローズ・クローズ（1取引日の終了時から別のオープン時まで）で計算されます。  
返品は、データがベンチマークに対して調整されていないことを意味する生のものであるか、市場全体の移動が計上されていることを意味する市場残存（Mktres）のいずれかである。  
戻り値は任意の間隔で計算できます。ここに1日と10日の視野があります。  
後ろ向きの場合は '前'、後ろ向きの場合は '次'でタグ付けされます。  
市場データ内には、次の列があります。  

time（datetime64 [ns、UTC]） - 現在の時刻（市場データでは、すべての行はUTCの22:00に取得されます）  
assetCode（オブジェクト） - アセットの一意のID  
assetName（category） -  assetCodesのグループに対応する名前。対応するassetCodeにニュースデータの行がない場合、これらは「不明」になる可能性があります。  
universe（float64） - その日の楽器がスコアリングに含まれるかどうかを示すブール値。この値はトレーニングデータ期間外には提供されません。与えられた日付の取引ユニバースは取引のために利用可能な一連の商品です（スコアリング機能は取引世界にない商品を考慮しません）。取引の世界は毎日変わります。  
volume（float64） - 当日の株式取引量  
close（float64） -  1日の終値（分割または配当の調整なし）  
open（float64） - 当日のオープンプライス（分割または配当の調整なし）  
returnsClo​​sePrevRaw1（float64） - 上記の返品説明を参照してください  
returnsOpenPrevRaw1（float64） - 上記の返品説明を参照  
returnsClo​​sePrevMktres1（float64） - 上記の返品説明を参照してください  
returnsOpenPrevMktres1（float64） - 上記の返品の説明を参照してください  
returnsClo​​sePrevRaw10（float64） - 上記の返品説明を参照してください  
returnsOpenPrevRaw10（float64） - 上記の返品の説明をご覧ください  
returnsClo​​sePrevMktres10（float64） - 上記の返品説明を参照してください  
returnsOpenPrevMktres10（float64） - 上記の返品の説明を参照  
returnsOpenNextMktres10（float64） -  10日、市場残存リターン。これは競争スコアリングで使用されるターゲット変数です。市場データは、  returnsOpenNextMktres10が常にnullでないようにフィルタリングされています。  

ニュースデータ  
ニュースデータには、ニュース記事レベルと資産レベルの両方の情報が含まれています（つまり、表は意図的に正規化されていません）。  

time（datetime64 [ns、UTC]） - データがフィードで使用可能になったときを示すUTCタイムスタンプ（秒精度）  
sourceTimestamp（datetime64 [ns、UTC]） - 作成されたこのニュース項目のUTCタイムスタンプ  
firstCreated（datetime64 [ns、UTC]） - アイテムの最初のバージョンのUTCタイムスタンプ  
sourceId（オブジェクト） - 各ニュース項目のID  
headline（オブジェクト） - アイテムの見出し  
urgency（int8） - ストーリーの種類を区別する（1：警告、3：記事）  
takeSequence（int16） - ニュース項目のテイクシーケンス番号.1から始まります。特定のストーリーの場合、アラートと記事は別々のシーケンスを持ちます。  
provider（カテゴリ） - ニュースアイテムを提供した組織の識別子（たとえば、ロイターニュースのRTRS、ビジネスワイヤのBSW）  
subjects（カテゴリ） - このニュースアイテムに関連するトピックコードおよび会社識別子。トピックコードは、ニュースアイテムの主題を記述します。これらは、資産クラス、地域、イベント、産業/セクター、および他のタイプをカバーすることができます。  
audiences（カテゴリ） - ニュースアイテムが属するデスクトップニュースプロダクトを識別します。彼らは通常、特定の視聴者に合わせて調整されています。 （例えば、マネーインターナショナルニュースサービスの場合は「M」、フランスの一般ニュースサービスの場合は「FB」）  

bodySize（int32） - 文字の現在のバージョンのストーリー本体のサイズ  
companyCount（int8） - 件名フィールドのニュース項目に明示的にリストされている会社の数  
headlineTag（オブジェクト） - ニュース項目のThomson Reuters見出しタグ  
marketCommentary（bool） - アイテムが「After the Bell」サマリなどの一般的な市況を議論していることを示すブール値インジケータ  
sentenceCount（int16） - ニュースアイテム内のセンテンスの総数。 firstMentionSentenceと組み合わせて、アイテムの最初の言及の相対的な位置を決定することができます。  
wordCount（int32） - ニュース項目の語彙トークン（単語と句読点）の合計数  
assetCodes（カテゴリ） - アイテムに記載されているアセットのリスト  
assetName（カテゴリ） - 資産の名前  
firstMentionSentence（int16） - スコアリングされたアセットが記述されている見出しから始まる最初のセンテンス。  
1：見出し  
2：物語の最初の文章  
3：体の2番目の文など  
0：スコアリング対象のアセットがニュースアイテムの見出しまたは本文に見つかりませんでした。その結果、ニュースアイテムのテキスト（見出し+本文）全体が感情スコアを決定するために使用されます。  
relevance（float32） - ニュース項目の資産への関連性を示す10進数。アセットが見出しに記載されている場合、関連性は1に設定されます。項目がアラート（緊急度== 1）の場合、関連性はfirstMentionSentenceによって測定される必要があります。  
sentimentClass（int8） - 資産に関するこのニュース項目の優勢なセンチメントクラスを示します。示されたクラスは、最も高い確率を有するクラスである。  
センチメントネガティブ（float32） - ニュース項目の感情が資産に対して負であった確率  
sentimentNeutral（float32） - ニュース項目の感情が資産に対して中立だった可能性  
sentimentPositive（float32） - ニュース項目の感情が資産に対してプラスであった可能性  
sentimentWordCount（int32） - アセットに関連すると見なされるアイテムテキストのセクション内のレキシカルトークンの数。これは、wordCountと組み合わせて使用​​して、資産を議論するニュース項目の割合を決定することができます。  
noveltyCount12H（int16） - 特定の資産に関するニュース項目内のコンテンツの12時間の新規性。これは、資産を含む以前のニュースアイテムのキャッシュ上の資産固有のテキストと比較することによって計算されます。  
noveltyCount24H（int16） - 上記と同じですが、24時間  
noveltyCount3D（int16） - 上記と同じですが、3日間  
noveltyCount5D（int16） - 上記と同じですが、5日間  
noveltyCount7D（int16） - 上記と同じですが、7日間  
volumeCounts12H（int16） - 各資産の12時間分のニュース。以前のニュース項目のキャッシュが維持され、5つの過去期間のそれぞれの中で資産を言及するニュース項目の数が計算される。  
volumeCounts24H（int16） - 上記と同じですが、24時間  
volumeCounts3D（int16） - 上記と同じですが、3日間  
volumeCounts5D（int16） - 上記と同じですが、5日間  
volumeCounts7D（int16） - 上記と同じですが、7日間  
Intrinioが提供する市場データ。  


In [2]:
# news_train_df = pd.read_csv("input/news_sample.csv")
# data_train_df = pd.read_csv("input/marketdata_sample.csv")

In [3]:
from kaggle.competitions import twosigmanews

In [4]:
env = twosigmanews.make_env()

Loading the data... This could take a minute.
Done!


In [5]:
(data_train_df, news_train_df) = env.get_training_data()

In [6]:
pd.set_option('display.max_columns', 50)

In [7]:
news_train_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,e58c6279551b85cf,China's Daqing pumps 43.41 mln tonnes of oil i...,3,1,RTRS,"{'ENR', 'ASIA', 'CN', 'NGS', 'EMRG', 'RTRS', '...","{'Z', 'O', 'OIL'}",1438,1,,False,11,275,"{'0857.HK', '0857.F', '0857.DE', 'PTR.N'}",PetroChina Co Ltd,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,2007-01-01 07:03:34+00:00,2007-01-01 07:03:34+00:00,5a31c4327427f63f,"FEATURE-In kidnapping, finesse works best",3,1,RTRS,"{'FEA', 'CA', 'LATAM', 'MX', 'INS', 'ASIA', 'I...","{'PGE', 'PCO', 'G', 'ESN', 'MD', 'PCU', 'DNP',...",4413,1,FEATURE,False,55,907,{'STA.N'},Travelers Companies Inc,8,0.447214,-1,0.600082,0.345853,0.054064,62,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,1cefd27a40fabdfe,PRESS DIGEST - Wall Street Journal - Jan 1,3,1,RTRS,"{'RET', 'ENR', 'ID', 'BG', 'US', 'PRESS', 'IQ'...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",2108,2,PRESS DIGEST,False,15,388,"{'WMT.DE', 'WMT.N'}",Wal-Mart Stores Inc,14,0.377964,-1,0.450049,0.295671,0.254280,67,0,0,0,0,0,0,0,5,11,17
3,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",1776,6,PRESS DIGEST,False,14,325,"{'GOOG.O', 'GOOG.OQ', 'GOOGa.DE'}",Google Inc,13,0.149071,-1,0.752917,0.162715,0.084368,83,0,0,0,0,0,0,0,5,13,15
4,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",1776,6,PRESS DIGEST,False,14,325,{'XMSR.O'},XM Satellite Radio Holdings Inc,11,0.149071,-1,0.699274,0.209360,0.091366,102,0,0,0,0,0,0,0,0,0,0


In [8]:
data_train_df.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672,1.0
1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803,0.0
2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433,1.0
3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425,1.0
4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994,1.0


In [9]:
data_train_df["oc"] = data_train_df.open - data_train_df.close

In [10]:
data_train_df["sell"] = data_train_df.volume/data_train_df.close

In [11]:
data_train_df.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe,oc,sell
0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672,1.0,-0.02,80984.777881
1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803,0.0,-0.04,184496.402878
2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433,1.0,0.48,31053.052519
3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425,1.0,1.49,280237.538353
4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994,1.0,-0.01,67069.922309


In [12]:
data_train_df.isnull().sum()

time                            0
assetCode                       0
assetName                       0
volume                          0
close                           0
open                            0
returnsClosePrevRaw1            0
returnsOpenPrevRaw1             0
returnsClosePrevMktres1     15980
returnsOpenPrevMktres1      15988
returnsClosePrevRaw10           0
returnsOpenPrevRaw10            0
returnsClosePrevMktres10    93010
returnsOpenPrevMktres10     93054
returnsOpenNextMktres10         0
universe                        0
oc                              0
sell                            0
dtype: int64

In [13]:
news_train_df.isnull().sum()

time                    0
sourceTimestamp         0
firstCreated            0
sourceId                0
headline                0
urgency                 0
takeSequence            0
provider                0
subjects                0
audiences               0
bodySize                0
companyCount            0
headlineTag             0
marketCommentary        0
sentenceCount           0
wordCount               0
assetCodes              0
assetName               0
firstMentionSentence    0
relevance               0
sentimentClass          0
sentimentNegative       0
sentimentNeutral        0
sentimentPositive       0
sentimentWordCount      0
noveltyCount12H         0
noveltyCount24H         0
noveltyCount3D          0
noveltyCount5D          0
noveltyCount7D          0
volumeCounts12H         0
volumeCounts24H         0
volumeCounts3D          0
volumeCounts5D          0
volumeCounts7D          0
dtype: int64

dataには会社名・日時・株の値段推移のデータがあります。  
newsはニュースのタイトルと概要が入ってます。  
subjectsなどは国などニュースに関わる国が複数入っています。  
学習しやすいように、リスト型に直しましょう。

In [14]:
news_train_df.headline.head()

0    China's Daqing pumps 43.41 mln tonnes of oil i...
1            FEATURE-In kidnapping, finesse works best
2           PRESS DIGEST - Wall Street Journal - Jan 1
3                PRESS DIGEST - New York Times - Jan 1
4                PRESS DIGEST - New York Times - Jan 1
Name: headline, dtype: object

In [15]:
# news_train_df['subjects'] = news_train_df['subjects'].str.findall(f"'([\w\./]+)'")
# news_train_df['audiences'] = news_train_df['audiences'].str.findall(f"'([\w\./]+)'")
# news_train_df['assetCodes'] = news_train_df['assetCodes'].str.findall(f"'([\w\./]+)'")
#news_train_df["headline"] = news_train_df["headline"].str.findall(f"'([\w\./]+)'")

今回はニュースを学習データにして予測していきたいので、newsにあるheadlineを学習できる形に持っていきましょう。  
まずは大文字を小文字にしスペースを区切っていきましょう。

In [16]:
news_train_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,e58c6279551b85cf,China's Daqing pumps 43.41 mln tonnes of oil i...,3,1,RTRS,"{'ENR', 'ASIA', 'CN', 'NGS', 'EMRG', 'RTRS', '...","{'Z', 'O', 'OIL'}",1438,1,,False,11,275,"{'0857.HK', '0857.F', '0857.DE', 'PTR.N'}",PetroChina Co Ltd,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,2007-01-01 07:03:34+00:00,2007-01-01 07:03:34+00:00,5a31c4327427f63f,"FEATURE-In kidnapping, finesse works best",3,1,RTRS,"{'FEA', 'CA', 'LATAM', 'MX', 'INS', 'ASIA', 'I...","{'PGE', 'PCO', 'G', 'ESN', 'MD', 'PCU', 'DNP',...",4413,1,FEATURE,False,55,907,{'STA.N'},Travelers Companies Inc,8,0.447214,-1,0.600082,0.345853,0.054064,62,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,1cefd27a40fabdfe,PRESS DIGEST - Wall Street Journal - Jan 1,3,1,RTRS,"{'RET', 'ENR', 'ID', 'BG', 'US', 'PRESS', 'IQ'...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",2108,2,PRESS DIGEST,False,15,388,"{'WMT.DE', 'WMT.N'}",Wal-Mart Stores Inc,14,0.377964,-1,0.450049,0.295671,0.254280,67,0,0,0,0,0,0,0,5,11,17
3,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",1776,6,PRESS DIGEST,False,14,325,"{'GOOG.O', 'GOOG.OQ', 'GOOGa.DE'}",Google Inc,13,0.149071,-1,0.752917,0.162715,0.084368,83,0,0,0,0,0,0,0,5,13,15
4,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",1776,6,PRESS DIGEST,False,14,325,{'XMSR.O'},XM Satellite Radio Holdings Inc,11,0.149071,-1,0.699274,0.209360,0.091366,102,0,0,0,0,0,0,0,0,0,0


In [17]:
data_train_df.shape

(4072956, 18)

# 自然言語処理
今回はニュースタイトルを基準にどのようなニュースが株に影響を与えるか見ていきたいと思います。
まずはニュースタイトルをベクトル化する為の準備をしていきましょう。

In [18]:
def get_wordCloud(corpus):
    wordCloud = WordCloud(background_color='white',
                              stopwords=STOPWORDS,
                              width=3000,
                              height=2500,
                              max_words=200,
                              random_state=42
                         ).generate(str(corpus))
    return wordCloud

In [19]:
def get_corpus(data):
    corpus = []
    for phrase in data:
        for word in phrase.split():
            corpus.append(word)
    return corpus

## タイトルの小文字化と記号の削除

In [20]:
# headline = news_train_df[:10000].headline.str.lower()

In [21]:
# headline = headline.str.replace(r"[^a-z1-9]"," ")

In [22]:
# headline.head()

## WordCloud
wordCloudで単語の出現頻度をかっこよく可視化する

In [23]:
# from wordcloud import WordCloud,STOPWORDS

In [24]:
import seaborn as sns
from scipy.stats import norm

In [25]:
# corpus = get_corpus(headline.values)

In [26]:
# procWordCloud = get_wordCloud(corpus)

In [27]:
# fig = plt.figure(figsize=(20, 8))
# plt.subplot(1,2,1)
# plt.imshow(procWordCloud)
# plt.axis('off')

# plt.subplot(1,2,2)

# words_count = [len(x.split(' ')) for x in headline.values]
# sns.distplot(words_count,hist=True, kde=False, bins=10, fit=norm)
# plt.title("Distribution of words in headline news")
# plt.xlabel('Number of words in headline news')

特にこれには意味がないですが、watch jan digestなどが多く使われていることがわかりました。

In [28]:
# headline = headline.str.split()

In [29]:
# headline.head()

In [30]:
# from gensim.models import Word2Vec,Doc2Vec
# from gensim.models.doc2vec import Doc2Vec
# from gensim.models.doc2vec import TaggedDocument

In [31]:
# import gensim

In [32]:
# headline = headline.str.split()

# Doc2Vecの学習データとモデルの作成

In [33]:
# trainings = [TaggedDocument(words = data.split(),tags = [i]) for i,data in enumerate(headline[:10000])]

In [34]:
# model = Doc2Vec(documents= trainings, dm = 1, size=20, window=8, min_count=0, workers=4,epochs=1000)

# model.save("doc2vec.model")

In [35]:
# model = Doc2Vec.load('doc2vec.model')

In [36]:
# model.docvecs.most_similar(42)

# 文字のベクトル化

In [37]:
# line_vec = []
# for i in headline.str.split():
#     line_vec.append(list(model.infer_vector(i)))

In [38]:
# line_vec = np.array(line_vec)

In [39]:
# line_vec

# SVD

In [40]:
# from sklearn.decomposition import TruncatedSVD

In [41]:
# svd = TruncatedSVD()
# nwe_line_vec = svd.fit_transform(line_vec)

In [42]:
# plt.scatter(nwe_line_vec[:,0],nwe_line_vec[:,1])

In [43]:
# plt.figure(figsize=[20,20])
# for i,v in enumerate(news_train_df.headline):
#     plt.annotate(v,(nwe_line_vec[i,0],nwe_line_vec[i,1]))
    
# plt.scatter(nwe_line_vec[:,0],nwe_line_vec[:,1])


# fig.show()

タイトルをベクトル化し表示させて見ました。  
なんとなく近しいタイトルは寄っている気がします。。。w

# news_train_dfで使う情報の抽出

In [44]:
news_train_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,e58c6279551b85cf,China's Daqing pumps 43.41 mln tonnes of oil i...,3,1,RTRS,"{'ENR', 'ASIA', 'CN', 'NGS', 'EMRG', 'RTRS', '...","{'Z', 'O', 'OIL'}",1438,1,,False,11,275,"{'0857.HK', '0857.F', '0857.DE', 'PTR.N'}",PetroChina Co Ltd,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,2007-01-01 07:03:34+00:00,2007-01-01 07:03:34+00:00,5a31c4327427f63f,"FEATURE-In kidnapping, finesse works best",3,1,RTRS,"{'FEA', 'CA', 'LATAM', 'MX', 'INS', 'ASIA', 'I...","{'PGE', 'PCO', 'G', 'ESN', 'MD', 'PCU', 'DNP',...",4413,1,FEATURE,False,55,907,{'STA.N'},Travelers Companies Inc,8,0.447214,-1,0.600082,0.345853,0.054064,62,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,1cefd27a40fabdfe,PRESS DIGEST - Wall Street Journal - Jan 1,3,1,RTRS,"{'RET', 'ENR', 'ID', 'BG', 'US', 'PRESS', 'IQ'...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",2108,2,PRESS DIGEST,False,15,388,"{'WMT.DE', 'WMT.N'}",Wal-Mart Stores Inc,14,0.377964,-1,0.450049,0.295671,0.254280,67,0,0,0,0,0,0,0,5,11,17
3,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",1776,6,PRESS DIGEST,False,14,325,"{'GOOG.O', 'GOOG.OQ', 'GOOGa.DE'}",Google Inc,13,0.149071,-1,0.752917,0.162715,0.084368,83,0,0,0,0,0,0,0,5,13,15
4,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",1776,6,PRESS DIGEST,False,14,325,{'XMSR.O'},XM Satellite Radio Holdings Inc,11,0.149071,-1,0.699274,0.209360,0.091366,102,0,0,0,0,0,0,0,0,0,0


In [45]:
news_col = ["time","urgency","takeSequence","companyCount","sentenceCount","wordCount"
            ,"firstMentionSentence","relevance","sentimentClass","sentimentNegative","sentimentNeutral",
            "sentimentPositive","sentimentWordCount","volumeCounts7D","assetName"]

In [46]:
news_train_df = news_train_df[news_col]

In [47]:
# marge_df["headline1"] = nwe_line_vec[:,0]
# marge_df["headline2"] = nwe_line_vec[:,1]

In [48]:
import pandas.tseries.offsets as offsets

In [49]:
times = pd.to_datetime(news_train_df['time'])

In [50]:
times = times+offsets.Hour(10)

In [51]:
times = times.astype("str").str.split(expand=True)[0]

In [52]:
news_train_df["time"] = times  

In [53]:
news_train_df.head()

,time,urgency,takeSequence,companyCount,sentenceCount,wordCount,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,volumeCounts7D,assetName
0,2007-01-01,3,1,1,11,275,6,0.235702,-1,0.500739,0.419327,0.079934,73,7,PetroChina Co Ltd
1,2007-01-01,3,1,1,55,907,8,0.447214,-1,0.600082,0.345853,0.054064,62,3,Travelers Companies Inc
2,2007-01-01,3,1,2,15,388,14,0.377964,-1,0.450049,0.295671,0.254280,67,17,Wal-Mart Stores Inc
3,2007-01-01,3,1,6,14,325,13,0.149071,-1,0.752917,0.162715,0.084368,83,15,Google Inc
4,2007-01-01,3,1,6,14,325,11,0.149071,-1,0.699274,0.209360,0.091366,102,0,XM Satellite Radio Holdings Inc


In [54]:
del times

In [55]:
news_train_df = news_train_df.groupby(["time","assetName"],as_index=False,observed=True).mean()

In [56]:
data_train_df["time"] = data_train_df.time.astype("str").str.split(expand=True)[0]

In [57]:
data_train_df = pd.merge(data_train_df,news_train_df,on=["assetName","time"],how="left")

In [58]:
market_train = data_train_df

In [59]:
market_train["time"] = pd.to_datetime(market_train['time'])

In [60]:
del data_train_df

In [61]:
import gc

gc.collect()

46

In [62]:
market_train.head()

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe,oc,sell,urgency,takeSequence,companyCount,sentenceCount,wordCount,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,volumeCounts7D
0,2007-02-01,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672,1.0,-0.02,80984.777881,3.000000,1.000000,1.000000,32.0,859.000000,1.0,1.000000,1.000000,0.043356,0.130069,0.826575,859.0,6.0
1,2007-02-01,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803,0.0,-0.04,184496.402878,1.857143,1.857143,2.142857,17.0,523.714286,1.0,1.000000,0.571429,0.055924,0.460296,0.483780,416.0,39.0
2,2007-02-01,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433,1.0,0.48,31053.052519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-02-01,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425,1.0,1.49,280237.538353,3.000000,1.000000,1.000000,66.0,1242.000000,1.0,1.000000,-1.000000,0.815723,0.127239,0.057038,116.0,42.0
4,2007-02-01,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994,1.0,-0.01,67069.922309,3.000000,1.000000,2.000000,7.0,112.000000,6.0,0.447214,0.000000,0.016462,0.861472,0.122067,60.0,4.0


In [63]:
market_train.shape

(4072956, 31)

In [64]:
del news_train_df

In [65]:
def print_varsize():
    import types
    print("{}{: >15}{}{: >10}{}".format('|','Variable Name','|','  Size','|'))
    print(" -------------------------- ")
    for k, v in globals().items():
        if hasattr(v, 'size') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(v.size),'|'))
        elif hasattr(v, '__len__') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(len(v)),'|'))

In [66]:
print_varsize()

|  Variable Name|      Size|
 -------------------------- 
|             In|        67|
|            Out|        13|
|       news_col|        15|
|   market_train| 126261636|


In [67]:
cat_cols = ['assetCode']
num_cols = ['volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevMktres1',
                    'returnsOpenPrevMktres1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10', 'returnsClosePrevMktres10',
                    'returnsOpenPrevMktres10',"urgency","takeSequence","companyCount","sentenceCount","wordCount",
            "firstMentionSentence","relevance","sentimentClass","sentimentNegative","sentimentNeutral","sentimentPositive",
            "sentimentWordCount","volumeCounts7D","oc","sell"]

In [68]:
from sklearn.model_selection import train_test_split

train_indices, val_indices = train_test_split(market_train.index.values,test_size=0.25, random_state=23)

In [69]:
def encode(encoder, x):
    len_encoder = len(encoder)
    try:
        id = encoder[x]
    except KeyError:
        id = len_encoder
    return id

encoders = [{} for cat in cat_cols]


for i, cat in enumerate(cat_cols):
    print('encoding %s ...' % cat, end=' ')
    encoders[i] = {l: id for id, l in enumerate(market_train.loc[train_indices, cat].astype(str).unique())}
    market_train[cat] = market_train[cat].astype(str).apply(lambda x: encode(encoders[i], x))
    print('Done')

embed_sizes = [len(encoder) + 1 for encoder in encoders] #+1 for possible unknown assets


encoding assetCode ... Done


In [70]:
from sklearn.preprocessing import StandardScaler
 
market_train[num_cols] = market_train[num_cols].fillna(market_train[num_cols].mean())
print('scaling numerical columns')

scaler = StandardScaler()

#col_mean = market_train[col].mean()
#market_train[col].fillna(col_mean, inplace=True)
scaler = StandardScaler()
market_train[num_cols] = scaler.fit_transform(market_train[num_cols])


scaling numerical columns


In [71]:
gc.collect()

21

In [72]:
def get_input(market_train, indices):
    X_num = market_train.loc[indices, num_cols].values
    X = {'num':X_num}
    for cat in cat_cols:
        X[cat] = market_train.loc[indices, cat_cols].values
    y = (market_train.loc[indices,'returnsOpenNextMktres10'] >= 0).values
    r = market_train.loc[indices,'returnsOpenNextMktres10'].values
    u = market_train.loc[indices, 'universe']
    d = market_train.loc[indices, 'time'].dt.date
    return X,y,r,u,d

# r, u and d are used to calculate the scoring metric
X_train,y_train,r_train,u_train,d_train = get_input(market_train, train_indices)
X_valid,y_valid,r_valid,u_valid,d_valid = get_input(market_train, val_indices)


In [73]:
from sklearn.metrics import accuracy_score

In [74]:
del market_train

In [75]:
import warnings
warnings.filterwarnings(action ='ignore',category = DeprecationWarning)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from functools import partial
from hyperopt import hp, fmin, tpe
algo = partial(tpe.suggest, n_startup_jobs=10)
def auto_turing(args):
    model = XGBClassifier(n_jobs = 4, n_estimators = args['n_estimators'],max_depth=6)
    model.fit(X_train['num'],y_train.astype(int))
    confidence_valid = model.predict(X_valid['num'])*2 -1
    score = accuracy_score(confidence_valid>0,y_valid)
    print(args,score)
    return -score
# space = {"n_estimators":hp.choice("n_estimators",range(10,50))}
# print(fmin)
# best = fmin(auto_turing, space, algo=algo,max_evals=30)
# print(best)

model = XGBClassifier(n_jobs = 4, n_estimators = 38,max_depth=6)
model1 = LGBMClassifier(n_jobs = 4, n_estimators = 38,max_depth=6)
model2 = CatBoostClassifier(depth=6,n_estimators=38)

model.fit(X_train['num'],y_train.astype(int),verbose=1)
model1.fit(X_train['num'],y_train.astype(int),verbose=1)
model2.fit(X_train['num'],y_train.astype(int),verbose=1)

confidence_valid = model.predict(X_valid['num'])*2 -1
confidence_valid1 = model1.predict(X_valid['num'])*2 -1
confidence_valid2 = model2.predict(X_valid['num'])*2 -1


confidence_valid = (confidence_valid + confidence_valid1 + confidence_valid2)/3
confidence_valid[confidence_valid > 0] = 1.0
confidence_valid[confidence_valid < 0] = -1.0

score = accuracy_score(confidence_valid>0,y_valid)
print(score)


0:	learn: 0.6897693	total: 846ms	remaining: 31.3s
1:	learn: 0.6886885	total: 1.65s	remaining: 29.7s
2:	learn: 0.6882292	total: 2.35s	remaining: 27.4s
3:	learn: 0.6879318	total: 3.08s	remaining: 26.2s
4:	learn: 0.6877662	total: 3.76s	remaining: 24.8s
5:	learn: 0.6876393	total: 4.45s	remaining: 23.7s
6:	learn: 0.6875261	total: 5.18s	remaining: 22.9s
7:	learn: 0.6874331	total: 5.82s	remaining: 21.8s
8:	learn: 0.6873856	total: 6.46s	remaining: 20.8s
9:	learn: 0.6873211	total: 7.19s	remaining: 20.1s
10:	learn: 0.6872544	total: 7.91s	remaining: 19.4s
11:	learn: 0.6871986	total: 8.71s	remaining: 18.9s
12:	learn: 0.6871517	total: 9.46s	remaining: 18.2s
13:	learn: 0.6871040	total: 10.2s	remaining: 17.5s
14:	learn: 0.6870572	total: 10.9s	remaining: 16.7s
15:	learn: 0.6870212	total: 11.6s	remaining: 15.9s
16:	learn: 0.6869787	total: 12.3s	remaining: 15.1s
17:	learn: 0.6869224	total: 13s	remaining: 14.4s
18:	learn: 0.6868800	total: 13.8s	remaining: 13.8s
19:	learn: 0.6868450	total: 14.4s	remaining

In [76]:
# calculation of actual metric that is used to calculate final score
r_valid = r_valid.clip(-1,1) # get rid of outliers. Where do they come from??
x_t_i = confidence_valid * r_valid * u_valid
data = {'day' : d_valid, 'x_t_i' : x_t_i}
df = pd.DataFrame(data)
x_t = df.groupby('day').sum().values.flatten()
mean = np.mean(x_t)
std = np.std(x_t)
score_valid = mean / std
print(score_valid)

0.494088510791375


In [77]:
days = env.get_prediction_days()

In [78]:
import time

In [79]:
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
predicted_confidences = np.array([])
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    print(n_days,end=' ')
    news_obs_df =  news_obs_df[news_col]
    times = pd.to_datetime(news_obs_df['time'])
    times = times+offsets.Hour(10)
    times = times.astype("str").str.split(expand=True)[0]
    news_obs_df["time"] = times  
    news_obs_df = news_obs_df.groupby(["time","assetName"],as_index=False,observed=True).mean()
    market_obs_df["time"] = market_obs_df.time.astype("str").str.split(expand=True)[0]
    market_obs_df = pd.merge(market_obs_df,news_obs_df,on=["assetName","time"],how="left")
    market_obs_df["time"] = pd.to_datetime(market_obs_df['time'])
    market_obs_df["oc"] =market_obs_df.open - market_obs_df.close
    market_obs_df["sell"] = market_obs_df.volume/market_obs_df.close
    
    
    t = time.time()

    market_obs_df['assetCode_encoded'] = market_obs_df[cat].astype(str).apply(lambda x: encode(encoders[i], x))

    market_obs_df[num_cols] = market_obs_df[num_cols].fillna(market_obs_df[num_cols].mean())
    market_obs_df[num_cols] = scaler.transform(market_obs_df[num_cols])
    X_num_test = market_obs_df[num_cols].values
    X_test = {'num':X_num_test}
    X_test['assetCode'] = market_obs_df['assetCode_encoded'].values
    
    prep_time += time.time() - t
    t = time.time()
    market_prediction = model.predict(X_test['num'])*2 -1
    market_prediction1 = model1.predict(X_test['num'])*2 -1
    market_prediction2 = model2.predict(X_test['num'])*2 -1
    
    market_prediction = (market_prediction + market_prediction1 + market_prediction2)/3
    market_prediction[market_prediction > 0] = 1.0
    market_prediction[market_prediction < 0] = -1.0
    
    #print(type(market_prediction))
    #print(type(predicted_confidences))
    predicted_confidences = np.concatenate((predicted_confidences, market_prediction))
    prediction_time += time.time() -t
    
    t = time.time()
    preds = pd.DataFrame({'assetCode':market_obs_df['assetCode'],'confidence':market_prediction})
    # insert predictions to template
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(predictions_template_df.mean()).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t

env.write_submission_file()
total = prep_time + prediction_time + packaging_time
print(f'Preparing Data: {prep_time:.2f}s')
print(f'Making Predictions: {prediction_time:.2f}s')
print(f'Packing: {packaging_time:.2f}s')
print(f'Total: {total:.2f}s')

1 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 27